In [1]:
import re
import os
import sys
import string
import pickle
import tempfile
import numpy  as np
import pandas as pd
from time import time
import tensorflow as tf
from nltk.util import ngrams
import tensorflow.contrib.rnn as rnn
import matplotlib.pyplot as plot

tf.logging.set_verbosity(tf.logging.ERROR)

In [2]:
# Configuration -
num_ex = 3000#0
prev_num = 3000
newfile= prev_num != num_ex
famTHR = 200*num_ex/500000
seqlenTHR = 1001 #3400
numgrams= 3 # for Glove
# min_after_dequeue = 10000*num_ex/500000
num_epochs=10
batch_size = 100#0
# dummy = "end end end end end"
runonce=0
alphabet =list(string.ascii_uppercase)
alphabet = ['0']+alphabet
vocab_size = len(alphabet)
vector_size = 100
filter_file='data/filtered_seqs_all.csv'
corpus_file='data/filtered_seq_corpus.txt'
glove_file='data/glove/vectors_pfam.txt'
logs_path = '/tmp/tensorflow/logs'
num_classes_file='uniprot_num_classes'

# filepath='processed_uniprot.tfrecord'

In [4]:
def printbuf(x):
    sys.stdout.write(x)
    sys.stdout.write('\r')


In [5]:
def save_obj(obj, name ,overwrite=1):
	filename='data/'+ name + '.pkl';
	if(overwrite!=1 and os.path.exists(name)):
		return [];
	with open(filename, 'wb') as f:
		pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        print('File saved to '+filename)

def load_obj(name):
	filename='data/'+ name + '.pkl';
	# if(not os.path.exists(name)):
	# 	return [];
	with open('data/' + name + '.pkl', 'rb') as f:
		print('Loaded file '+filename)
		return pickle.load(f)

names = ['tri']
for i in range(100):
    names+= ['dim_'+str(i)]
Glove = pd.read_table(glove_file,sep = ' ',header=None,index_col='tri',names=names)
def genGlove(seq,n=3):
    grams = ngrams(list(seq),n)
#     grams = ngrams(eval(seq),n)
    return map(lambda tri:list(Glove.loc[tri]),[''.join(g) for g in grams])
#     return map(lambda tri:list(Glove.loc[tri]),[''.join(map(lambda x :alphabet[x],g)) for g in grams])

def saveFilteredData():
    fam='Cross-reference (Pfam)'
    seq = 'Sequence'
    main =pd.read_table("./data/uniprot-all.tab.gz", sep='\t',nrows=num_ex)
    main[fam]= main[fam].apply(lambda x:str(x).split(';')[0])
    col = (main[fam]!='nan') & (main[fam].isnull()==False) & (main[seq].isnull()==False)
    main = main[col]
#     length threshold
    main = main[main['Length'].astype(int) < seqlenTHR]
    
    alphafam = list(main[fam].unique())
    alphafam = [-1]+alphafam
    num_classes = len(alphafam)
    seq_records = main[[seq,fam,'Length']]
    seq_records.columns=['seq','fam','length']
    #All the heavy jobs here - takes upto 92 seconds
#     seq_records['seq']= seq_records['seq'].apply(lambda seq : list(map(lambda x : alphabet.index(x),list(seq))))
    
    seq_records['fam']= seq_records['fam'].apply(lambda x : alphafam.index(x))
    counts = seq_records.groupby(['fam']).size().reset_index(name='counts')
    #10801 -> 9254 families
    counts = counts[ (counts['counts']>=famTHR) & (counts['counts'] < 3400) ]
    #608 -> 555 families having count between 200 and 3400
    filtered_fams = list(counts['fam'])
    #323162 -> 302907 examples total satisfy it.
    filtered_seqs = seq_records[ seq_records['fam'].isin(filtered_fams)]
    filtered_seqs = filtered_seqs.sort_values('length')
    
    filtered_seqs['seq'] = filtered_seqs['seq'].apply(lambda x:genGlove(x))
#     Save for future use
    save_obj(num_classes,num_classes_file)
    filtered_seqs.to_csv(filter_file,index=False)
    print('Filtered Seqs Saved to : '+filter_file)

    # # takes 3 seconds !
    # gb = seq_records.groupby('fam')    
    # family_wise_db = [gb.get_group(x) for x in gb.groups]

if newfile or not os.path.exists(filter_file):
    newfile=False
    print('newFile or File not found, preparing data again')
    p = time()
    saveFilteredData()
    print('time taken: ',time() - p)

num_classes=load_obj(num_classes_file)
p = time()
filtered_seqs = pd.read_csv(filter_file,nrows=num_ex)
print("Loaded filtered_seqs")
print('time taken: ',time() - p)


Loaded file data/uniprot_num_classes.pkl
Loaded filtered_seqs
('time taken: ', 10.306251049041748)


In [6]:
filtered_seqs.describe()

,fam,length
count,1486.000000,1486.000000
mean,106.561911,215.010094
std,59.769477,101.906628
min,1.000000,32.000000
25%,64.500000,122.000000
50%,105.000000,205.000000
75%,145.000000,315.000000
max,274.000000,375.000000


In [11]:
def pad(x,max_len):
    xcp=eval(x) if(type(x)==str) else x #For further epochs
    try :
        return np.lib.pad(xcp,pad_width=[(0,max_len - len(xcp)),(0,0)],mode='constant',constant_values=(-1,0))    
    except:
        print(x)
        print("Oops!",sys.exc_info()[0],"occured.")
        return np.lib.pad(xcp,pad_width=[(0,max_len - len(xcp)),(0,0)],mode='constant',constant_values=(-1,0))    
#     return np.lib.pad(x,(0,max_len - len(x)),'constant',constant_values=(-1,0))
class SimpleDataIterator():
    def __init__(self, df):
        self.df = df
        self.size = len(self.df)
        self.epochs = 0
        self.batch_no = 0
        self.shuffle()

    def shuffle(self):
        self.df = self.df.sample(frac=1).reset_index(drop=True)
        self.cursor = 0

class PaddedDataIterator(SimpleDataIterator):
    def next_batch(self, n):
        if self.cursor + n > self.size:
            self.epochs += 1
            self.batch_no=0
            self.shuffle()#resets the cursor as well.
        
        res = self.df.iloc[self.cursor:self.cursor+n]
        self.cursor += n
        self.batch_no+=1

        # Pad sequences with 0s so they are all the same length
        maxlen = max(res['length'])
        res2 = res.copy()
        res2['seq'] = map(lambda x:pad(x,maxlen),res2['seq'])
        return res2



In [12]:
class RnnForPfcModelOne:
#   @profile
    def __init__(self, 
        num_classes = 549, 
        hidden_units=100,
        learning_rate=0.01,
         logs_path='/tmp/tensorflow/logs'
                ):
        global vocab_size
        global vector_size
        # batch_size * no_of_time_steps * vocab_size _/
        self.weights = tf.Variable(tf.random_uniform(shape=[hidden_units, num_classes], maxval=1),name="Weights")
        self.biases = tf.Variable(tf.random_uniform(shape=[num_classes]),name="Biases")
        self.rnn_fcell = rnn.BasicLSTMCell(num_units = hidden_units, 
                                           forget_bias = 1.0,
                                           activation = tf.tanh)
        """
        We now have the glove file, containing vector spaces for all of the trigrams we have in vocab. So now rather than passing 
        the one-hot vectors, we'll pass the GloVes having reduced dimensions.
        """

        # self.len_data taken from feed_dict
        self.len_data = tf.placeholder(tf.uint8, [None],name="Lengths")
        # self.x_input taken from feed_dict - now it'll be a GloVe
        self.x_input_g = tf.placeholder(tf.float32, [None, None,vector_size], name = 'X_GloVe') # batch_size * no_of_time_steps * vector_size
#         self.x_input = tf.placeholder(tf.uint8, [None, None], name = 'x_ip') # batch_size * no_of_time_steps _/
#         # self.x_input_o takes self.x_input
#         self.x_input_o = tf.one_hot(indices = self.x_input, 
#             depth = vocab_size,
#             on_value = 1.0,
#             off_value = 0.0,
#             axis = -1)
        # self.outputs takes self.x_input_o & len_data
        with tf.name_scope('DynamicRNN'):
            self.outputs, self.states = tf.nn.dynamic_rnn(self.rnn_fcell,
                                                      self.x_input_g,
#                                                       self.x_input_o,
                                                      sequence_length = self.len_data,
                                                      dtype = tf.float32)
        
        # outputs of shape batch_size * no_of_time_steps * vocab_size
        # output at time t i.e. the last output
        self.outputs_t = tf.reshape(self.outputs[:, -1, :], [-1, hidden_units])
        # The single layer NN to classify - takes outputs_t
        self.y_predicted = tf.matmul(self.outputs_t, self.weights) + self.biases
        
        
        # self.y_input taken from feed_dict batch_size *1
        self.y_input = tf.placeholder(tf.uint8, [None], name = 'y_ip')
        # self.y_input_o takes y_input
        self.y_input_o = tf.one_hot(indices = self.y_input, 
                                    depth = num_classes,
                                    on_value = 1.0,
                                    off_value = 0.0,
                                    axis = -1)
        
        # self.loss takes one hot y and y_predicted
        with tf.name_scope('CrossEntropy'):
            self.loss = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(logits=self.y_predicted, labels=self.y_input_o)
            )
        #y_predicted and y_input_o shud be of same size = batch_size * num_classes
        # define optimizer and trainer
        with tf.name_scope('AdamOptimizer'):
            self.trainer = tf.train.AdamOptimizer(learning_rate).minimize(self.loss)
#             self.trainer = tf.train.GradientDescentOptimizer(learning_rate).minimize(self.loss)

        self.sess = tf.Session()
        self.sess.run(tf.global_variables_initializer())

        self.get_equal = tf.equal(tf.argmax(self.y_input_o, 1), tf.argmax(self.y_predicted, 1))
        self.accuracy = tf.reduce_mean(tf.cast(self.get_equal, tf.float32),name="Accuracy")
        
        self.summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
        # Create a summary to monitor cost tensor
        tf.summary.scalar("loss", self.loss)
        # Create a summary to monitor accuracy tensor
        tf.summary.scalar("accuracy", self.accuracy)
        # Merge all summaries into a single op - to pass into sess.run
        self.merged_summary_op = tf.summary.merge_all()

#   @profile
    def predict(self, x, y, len_data):
        result = self.sess.run(self.y_predicted, feed_dict={self.x_input_g: x, self.y_input: y, self.len_data:len_data})
        #result = self.sess.run(self.y_predicted, feed_dict={self.x_input: x, self.y_input: y, self.len_data:len_data})
        return result

#   @profile
    def optimize(self, x, y, len_data,summary_index):
#         c, summary = self.sess.run([self.loss, self.merged_summary_op],
        c = self.sess.run([self.loss],
                              feed_dict={self.x_input_g: x, self.y_input: y, self.len_data:len_data})
                              #feed_dict={self.x_input: x, self.y_input: y, self.len_data:len_data})
        
        
#         self.summary_writer.add_summary(summary, summary_index)

        self.sess.run(self.trainer, feed_dict={self.x_input_g: x, self.y_input: y, self.len_data:len_data})
        #self.sess.run(self.trainer, feed_dict={self.x_input: x, self.y_input: y, self.len_data:len_data})
        return c

#   @profile
    def cross_validate(self, x, y, len_data):
        result = self.sess.run(self.accuracy, feed_dict={self.x_input_g:x, self.y_input:y, self.len_data:len_data})
        #result = self.sess.run(self.accuracy, feed_dict={self.x_input:x, self.y_input:y, self.len_data:len_data})
        return result

#   @profile
    def close_summary_writer(self):
        self.summary_writer.close()

In [13]:
if(not runonce):
    runonce=1
    model = RnnForPfcModelOne(num_classes=num_classes)
    
it = PaddedDataIterator(filtered_seqs)    
while(it.epochs != num_epochs):
    df = it.next_batch(batch_size)
    batch_x, batch_y, len_data = map(list,(df['seq'],df['fam'],df['length']))
#     batch_x of shape batch_size * max_len * vector_size
    
    cost = model.optimize(batch_x, batch_y, len_data,it.batch_no + batch_size*it.epochs)
    accuracy_known = model.cross_validate(batch_x,batch_y,len_data)
    print(it.epochs, it.batch_no,'Cost :',cost,"Training acc : %d" % (accuracy_known) )

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(0, 1, 'Cost :', [5.0616164], 'Training acc : 0')
(0, 2, 'Cost :', [5.1055784], 'Training acc : 0')
(0, 3, 'Cost :', [5.0535107], 'Training acc : 0')
(0, 4, 'Cost :', [5.0147557], 'Training acc : 0')
(0, 5, 'Cost :', [4.9840946], 'Training acc : 0')
(0, 6, 'Cost :', [5.0584993], 'Training acc : 0')
(0, 7, 'Cost :', [5.0251369], 'Training acc : 0')
(0, 8, 'Cost :', [5.0146003], 'Training acc : 0')
(0, 9, 'Cost :', [4.9490094], 'Training acc : 0')
(0, 10, 'Cost :', [4.9155455], 'Training acc : 0')
(0, 11, 'Cost :', [4.9769063], 'Training acc : 0')
(0, 12, 'Cost :', [4.9440641], 'Training acc : 0')
(0, 13, 'Cost :', [4.8957925], 'Training acc : 0')
(0, 14, 'Cost :', [4.911448], 'Training acc : 0')
[[ 0.478664  0.194855  0.268756 ..., -0.001515 -0.385026 -0.380525]
 [-0.266599 -0.016745  0.140885 ..., -0.044535  0.383963 -0.04804 ]
 [-0.112923 -0.150274  0.282372 ...,  0.278006 -0.252773  0.280834]
 ..., 
 [ 0.        0.        0.       ...,  0.        0.        0.      ]
 [ 0.        0.  

ValueError: [(0, -2), (0, 0)] cannot contain negative values.

In [ ]:
# A function that generates words of 3 grams from the given list of sequences
# Then returns without disturbing their 'context'
def func(x):
    if(x <0 or x>len(alphabet)):
        print("IndeX",x)
    print(x)
    return alphabet[x]

def genCorpus(filtered_seqs,n=3):
#     seq is already a string(list of integers) -
    total = filtered_seqs['seq'].size    
    print('generating corpus : ')
    with open(corpus_file,'a') as f:
        i =0
        for rec in filtered_seqs.iterrows():
            i+=1
            grams = ngrams(list(rec[1].seq),n)
            f.write(' '.join([''.join(g) for g in grams]))
#             grams = ngrams(eval(rec[1].seq),n)
#             f.write(' '.join([''.join(map(lambda x :alphabet[x],g)) for g in grams]))
            f.write(' ')
            perc = (i*100)/total
            printbuf('%d %% completed' % perc)
    print("Wrote corpus to file")

# if os.path.exists(corpus_file):
#     with open(corpus_file,'r') as f:
#         corpus = f.read()
#         print('loaded corpus')
# else :
#     p = time()
#     with open(corpus_file,'w') as f:
#         f.write('')
#     genCorpus(filtered_seqs,numgrams)
#     print('time taken: ',time() - p)